# Model Description

For $n=1,...,N$,
\begin{align}
\beta &\sim N(0,I)\\ 
Y_n &\sim \text{Bernoulli}\left(\frac{1}{1+\exp(-\beta^T x_n)}\right)
\end{align}.
This models the binary outcome $y_n$'s using the covariate $x_n$'s.

# Import Packages and Dataset

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt

In [3]:
# import breast cancer dataset for binary classification
x,y = load_breast_cancer(return_X_y=True)
dim_x = x.shape[1]

# Utilities

In [4]:
tf_float_type = tf.float64
np_float_type = np.float64
as_tf_float = lambda x: tf.cast(x, tf_float_type)

In [5]:
def tf_logsumexp(ary, axis=1, keepdims=False):
    return tf.math.reduce_logsumexp(ary, axis=axis, keepdims=keepdims)

def tf_logmeanexp(ary, axis=1, keepdims=False):
    return tf.math.reduce_logsumexp(ary, axis=axis, keepdims=keepdims) \
        - tf.math.log(as_tf_float(ary.shape[axis]))

# (Non-Bayesian) Logistic Regression on Dataset

In [6]:
m = LogisticRegression(max_iter=10000).fit(x,y)
score = m.score(x,y)

In [7]:
print('R2 score: {}'.format(score))

R2 score: 0.9578207381370826


# Bayesian Logistic Regression on dataset

## Variatioal Bayes by KL Divergence
Our estimator of $q(\beta)$ (⇔`loc_beta` and `scale_beta`) are obtained as:
$$\hat q_{\text{KL}} = \arg\min \mathrm{KL}[q(\beta)||p(\beta|y_{1:N};x_{1:N})],$$
where KL divergence is defined as 
$$\mathrm{KL}[q(x)||p(x)] = \mathrm{E}_{X\sim q(x)}\left[\log\frac{q(X)}{p(X)}\right].$$

In [8]:
def pointwise_ELBO(loc_beta, scale_beta, beta):
    """
    Compute ELBO for n_MC samples of beta, where the shape of beta is [n_MC, dim_x].
    
    Arguments:
    loc_beta: 1-d array of size [dim_x]
    scale_beta: 1-d array of size [dim_x]
    beta: 2-d array of size [n_MC, dim_x] representing the n_MC samples of beta taken from q_beta
    
    Returns:
    elbo: n_MC MC-samples of elbo
    """
    p_beta = tfp.distributions.Normal(loc=np.zeros([dim_x]), scale=np.ones([dim_x]))
    q_beta = tfp.distributions.Normal(loc=loc_beta, scale=scale_beta)
    z = beta @ x.T# of shape [n_MC, n_train]
    p_y = tfp.distributions.Bernoulli(logits=z)
    y_ = y.reshape([1, -1])
    elbo = tf.reduce_sum( p_y.log_prob(y_), axis=1)\
            + tf.reduce_sum( p_beta.log_prob(beta), axis=1)\
            - tf.reduce_sum( q_beta.log_prob(beta), axis=1)
    return elbo

In [9]:
def ELBO(loc_beta, scale_beta, n_MC=1):
    q_beta = tfp.distributions.Normal(loc=loc_beta, scale=scale_beta)
    beta = q_beta.sample(n_MC)# of shape [n_MC, dim_x]
    elbo = tf.reduce_mean( pointwise_ELBO(loc_beta, scale_beta, beta) )
    return elbo

In [10]:
loc = tf.Variable(np.zeros([dim_x]))
invSPscale = tf.Variable(np.ones([dim_x])) # defined as inv_softplus(scale) to keep the positive constraint of scale
    
for t in range(4000):
    rho_t = 5e-6/(10+t)**0.7
    
    with tf.GradientTape() as g:
        g.watch([loc, invSPscale])
        scale = tf.math.softplus(invSPscale) 
        score = ELBO(loc, scale, 100)
    dloc, dinvSPscale = g.gradient(score, [loc, invSPscale])

    loc = loc + rho_t*dloc
    invSPscale = invSPscale + rho_t*dinvSPscale
print(score)

tf.Tensor(-19761.456766640746, shape=(), dtype=float64)


In [11]:
score = np.mean((x @ loc.numpy()>0)==y) # accuracy of binary prediction
print('R2 score by plug-in prediction: {}'.format(score))

R2 score by plug-in prediction: 0.9068541300527241


## Variational Bayes by Renyi (or $\chi$) Divergence
Our estimator of $q(\beta)$ (⇔`loc_beta` and `scale_beta`) are obtained as:
$$\hat q_{\text{KL}} = \arg\min \mathrm{D}_{1/2}[q(\beta)||p(\beta|y_{1:N};x_{1:N})],$$
where Renyi (or $\chi$) divergence is defined as 
$$\mathrm{D}_\gamma[q(x)||p(x)] = \frac{1}{\gamma}\log\mathrm{E}_{X\sim q(x)}\left(\frac{p(X)}{q(X)}\right)^{\gamma}.$$
When $\gamma=1-\alpha$ for $\alpha>0$, above quantity becomes Renyi's $\alpha$-divergence. When $\gamma>1$, $-\mathrm{D}_\gamma[q(x)||p(x)]$ becomes $\chi$-divergence.

In [12]:
def pointwise_Renyi_Chi(loc_beta, scale_beta, beta, gamma):
    """
    Compute ELBO for n_MC samples of beta, where the shape of beta is [n_MC, dim_x].
    
    Arguments:
    loc_beta: 1-d array of size [dim_x]
    scale_beta: 1-d array of size [dim_x]
    beta: 3-d array of size [n_MC_out, n_MC_in, dim_x] representing the n_MC samples of beta taken from q_beta
    
    Returns:
    elbo: n_MC MC-samples of elbo
    """

    p_beta = tfp.distributions.Normal(loc=np.zeros([dim_x]), scale=np.ones([dim_x]))
    q_beta = tfp.distributions.Normal(loc=loc_beta, scale=scale_beta)
    z = beta @ x.T# of shape [n_MC_out, n_MC_in, n_train]
    p_y = tfp.distributions.Bernoulli(logits=z)
    log_prob_ratio = tf.reduce_sum( p_y.log_prob(y), axis=2)\
                    + tf.reduce_sum( p_beta.log_prob(beta), axis=2)\
                    - tf.reduce_sum( q_beta.log_prob(beta), axis=2)# of shape [n_MC_out, n_MC_in]
    score = 1./gamma * tf_logmeanexp(gamma * log_prob_ratio, axis=1)
    return score

In [13]:
def Renyi_Chi(loc_beta, scale_beta, gamma, n_MC_out=1, n_MC_in=1):
    q_beta = tfp.distributions.Normal(loc=loc_beta, scale=scale_beta)
    beta = q_beta.sample([n_MC_out, n_MC_in])# of shape [n_MC_out, n_MC_in, dim_x]
    score = tf.reduce_mean( pointwise_Renyi_Chi(loc_beta, scale_beta, beta, gamma) )
    return score

In [14]:
loc = tf.Variable(np.zeros([dim_x]))
invSPscale = tf.Variable(np.ones([dim_x])) # defined as inv_softplus(scale) to keep the positive constraint of scale
    
for t in range(4000):
    rho_t = 1e-4/(10+t)**0.7
    
    with tf.GradientTape() as g:
        g.watch([loc, invSPscale])
        scale = tf.math.softplus(invSPscale) 
        score = Renyi_Chi(loc, scale, gamma=1./2, n_MC_out=100, n_MC_in=20)
    dloc, dinvSPscale = g.gradient(score, [loc, invSPscale])

    loc = loc + rho_t*dloc
    invSPscale = invSPscale + rho_t*dinvSPscale
print(score)

tf.Tensor(-1431.275781856723, shape=(), dtype=float64)


In [15]:
score = np.mean((x @ loc.numpy()>0)==y) # accuracy of binary prediction
print('R2 score by plug-in prediction: {}'.format(score))

R2 score by plug-in prediction: 0.9103690685413005
